In [1]:
import os
import numpy as np
from time import time

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D ,Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [3]:
ROOT_DIR = os.path.abspath("../../")
DS_DIR = os.path.join(ROOT_DIR, "Datasets/VGGFEar")
DS_train_DIR = os.path.join(DS_DIR, "train")
DS_test_DIR = os.path.join(DS_DIR, "val")
img_width, img_height = 224, 224

In [4]:
trdata = ImageDataGenerator(
    rescale=1./255.)
traindata = trdata.flow_from_directory(
    directory=DS_train_DIR,
    target_size=(img_width,img_height),
    batch_size=32,
    class_mode='categorical')

tsdata = ImageDataGenerator(
    rescale=1./255.)
testdata = tsdata.flow_from_directory(
    directory=DS_test_DIR, 
    target_size=(img_width,img_height),
    batch_size=32,
    class_mode='categorical')

Found 240000 images belonging to 400 classes.
Found 80000 images belonging to 400 classes.


In [5]:
set_size = 400   #clases

In [6]:
vgg_conv = VGG16(weights='imagenet', include_top=True, input_shape=(img_width, img_height, 3))
vgg_conv.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
vgg_conv = Model(vgg_conv.input, vgg_conv.get_layer('flatten').output)

# Create the model
my_vgg = Sequential()
 
# Add the vgg convolutional base model
my_vgg.add(vgg_conv)
 
my_vgg.add(Dense(512, activation='relu', name='fc1'))
#my_vgg.add(Dropout(0.5))
my_vgg.add(Dense(512, activation='relu', name='fc2'))
#my_vgg.add(Dropout(0.5))
my_vgg.add(Dense(set_size, activation='softmax', name='predictions'))
 
# Show a summary of the model. Check the number of trainable parameters
my_vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_1 (Functional)    (None, 25088)             14714688  
_________________________________________________________________
fc1 (Dense)                  (None, 512)               12845568  
_________________________________________________________________
fc2 (Dense)                  (None, 512)               262656    
_________________________________________________________________
predictions (Dense)          (None, 400)               205200    
Total params: 28,028,112
Trainable params: 28,028,112
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Freeze the layers except the last 3 layers
for layer in my_vgg.layers[:1]:
    layer.trainable = False

for layer in my_vgg.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.functional.Functional object at 0x000001E275AD4D08> False
<tensorflow.python.keras.layers.core.Dense object at 0x000001E274978448> True
<tensorflow.python.keras.layers.core.Dense object at 0x000001E20035CE48> True
<tensorflow.python.keras.layers.core.Dense object at 0x000001E200362848> True


In [9]:
my_vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_1 (Functional)    (None, 25088)             14714688  
_________________________________________________________________
fc1 (Dense)                  (None, 512)               12845568  
_________________________________________________________________
fc2 (Dense)                  (None, 512)               262656    
_________________________________________________________________
predictions (Dense)          (None, 400)               205200    
Total params: 28,028,112
Trainable params: 13,313,424
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
my_vgg.compile(optimizer=Adam(lr=0.01), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])  #vgg16_1.h5--> 58%
#model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 99%  #with 2 dropout layers
#my_vgg.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 99.37%  #without dropout layers
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 98.75%
#model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 55%
#my_vgg.compile(optimizer=RMSprop(lr=0.001), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])  #vgg16_3.h5--> 55%

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    "vgg16_vggfaceeartrain400_trainclassif_OpAdam_100epoch.h5", 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', 
    period=1)

early = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0, 
    patience=15,
    #if it doesn’t see any rise in validation accuracy in 25,  the model will stop 
    verbose=1, 
    mode='auto')

start = time()
hist = my_vgg.fit_generator(
    steps_per_epoch=240000//32,
    generator=traindata, 
    validation_data= testdata, 
    validation_steps=80000//32,
    epochs=100,
    callbacks=[checkpoint,early])

print("TIME: {}".format((time()-start)/60))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
7500/7500 [==============================] - ETA: 0s - loss: 6.0393 - accuracy: 0.0024
Epoch 00001: val_accuracy improved from -inf to 0.00250, saving model to vgg16_vggfaceeartrain400_trainclassif_OpAdam_100epoch.h5
7500/7500 [==============================] - 982s 131ms/step - loss: 6.0393 - accuracy: 0.0024 - val_loss: 5.9997 - val_accuracy: 0.0025
Epoch 2/100
7500/7500 [==============================] - ETA: 0s - loss: 6.0013 - accuracy: 0.0024
Epoch 00002: val_accuracy did not improve from 0.00250
7500/7500 [==============================] - 852s 114ms/step - loss: 6.0013 - accuracy: 0.0024 - val_loss: 6.0003 - val_accuracy: 0.0025
Epoch 3/100
6961/7500 [==========================>...] - ETA: 46s - loss: 6.0012 - accuracy: 0.0024

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()
fig.savefig('vgg16_vggfaceeartrain400_trainclassif_OpAdam_100epoch.png') # Plot 4dataset merge training val_acc = 0.96875
plt.close()